# Avocado Data Analysis Notebook
ALT-TAB LABS LLP &copy; 2020 All Rights Reserved

In [1]:
# show files in current directory
import os
os.listdir()

['avocado_notebook.ipynb',
 'avocado-format.png',
 '__pycache__',
 'pyctfsglib.py',
 'avocado-test.csv',
 '.ipynb_checkpoints',
 'avocado-submission.csv',
 'avocado_notebook-experiment.ipynb',
 'avocado-train.csv']

In [2]:
# load csvs
import pandas as pd
#MyDataFrame = pd.read_csv('PATH_TO_CSV.csv', index_col='id')
AvoTrain = pd.read_csv('avocado-train.csv', index_col='id')

In [3]:
#View the data!
AvoTrain.head()
#AvoTrain.describe()

,DataBatch,Date,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region,AveragePrice
id,,,,,,,,,,,,,
0,7,2016-11-06,183542.31,98949.98,22891.61,95.00,61605.72,43571.99,17499.01,534.72,conventional,NewOrleansMobile,1.49
1,24,2017-07-16,224434.92,42951.31,120360.02,131.85,60991.74,53141.81,3621.04,4228.89,conventional,HarrisburgScranton,1.38
2,51,2015-01-04,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,Atlanta,1.76
3,22,2015-07-26,91825.07,1679.28,45615.48,741.77,43788.54,43788.54,0.00,0.00,conventional,BuffaloRochester,1.39
4,50,2015-01-11,54644.32,1491.88,33759.12,1325.17,18068.15,12165.94,5902.21,0.00,conventional,Pittsburgh,1.54


In [4]:
# Clean Data
import datetime as dt

# Drops missing values 
AvoTrain.dropna()

# Convert strings to numbers
AvoTrain.Date = [dt.date.fromisoformat(x).toordinal() for x in AvoTrain.Date]

AvoTypes = {
    "conventional": 0,
    "organic": 1
}
AvoTrain.type = [AvoTypes[x] for x in AvoTrain.type]

AvoRegion = {region: regionid for regionid, region in enumerate(set(AvoTrain.region))}
AvoTrain.region = [AvoRegion[x] for x in AvoTrain.region]

# Read more advanced ways to clean data
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [5]:
# Select data for learning
features = ["Date", "TotalVolume", "DataBatch", "type", "4046", "4225", "4770", "TotalBags", 
            "SmallBags", "LargeBags", "XLargeBags", "type", "region"]

X = AvoTrain[features] 
y = AvoTrain.AveragePrice

In [6]:
X.head()

,Date,TotalVolume,DataBatch,type,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region
id,,,,,,,,,,,,,
0,736274,183542.31,7,0,98949.98,22891.61,95.00,61605.72,43571.99,17499.01,534.72,0,27
1,736526,224434.92,24,0,42951.31,120360.02,131.85,60991.74,53141.81,3621.04,4228.89,0,0
2,735602,3846.69,51,1,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,1,50
3,735805,91825.07,22,0,1679.28,45615.48,741.77,43788.54,43788.54,0.00,0.00,0,53
4,735609,54644.32,50,0,1491.88,33759.12,1325.17,18068.15,12165.94,5902.21,0.00,0,6


In [7]:
y.head()

id
0    1.49
1    1.38
2    1.76
3    1.39
4    1.54
Name: AveragePrice, dtype: float64

# Regression Time!

In [8]:
# Sci Kit Learn! Scientific Computing library for python
from sklearn import *

In [9]:
# Pick the regression model we want to use

# Decision Tree Regression
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
# model = tree.DecisionTreeRegressor(random_state=2020)

# Random Forest Regression
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
model = ensemble.RandomForestRegressor(random_state=None, n_estimators=1000)

In [10]:
# Split training into some for training and some for testing
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.25)

print("training split: ", len(Xtrain), "; test split: ", len(Xtest))



training split:  9580 ; test split:  3194


In [11]:
# Perform regression on the data
model.fit(Xtrain, ytrain) # Xtrain, ytrain  for training X, y after you sure you not overfitting

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [12]:
# Score model
model.score(Xtest, ytest)

0.8948761307203282

# Predict for Testing data

In [13]:
# Load Data
AvoTest = pd.read_csv('avocado-test.csv', index_col='id')
AvoTest.head()

,DataBatch,Date,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region
id,,,,,,,,,,,,
0,21,2016-07-31,28969.34,80.77,27361.91,0.00,1526.66,1526.66,0.00,0.00,organic,Chicago
1,19,2017-08-20,322962.89,5632.85,242365.82,2441.56,72522.66,22942.48,46083.51,3496.67,conventional,NorthernNewEngland
2,49,2017-01-22,155334.45,3657.79,74068.65,0.00,77608.01,27302.29,50305.72,0.00,organic,GreatLakes
3,12,2015-10-04,10231.74,341.89,8519.00,0.00,1370.85,303.33,1067.52,0.00,organic,CincinnatiDayton
4,10,2015-10-18,10652.02,6905.95,39.52,0.00,3706.55,3706.55,0.00,0.00,organic,Houston


In [14]:
# Clean Data

# Drops missing values 
AvoTest.dropna()

# Convert strings to numbers
AvoTest.Date = [dt.date.fromisoformat(x).toordinal() for x in AvoTest.Date]

AvoTest.type = [AvoTypes[x] for x in AvoTest.type]

AvoTest.region = [AvoRegion[x] for x in AvoTest.region]

# Read more advanced ways to clean data
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [15]:
# Select features
XforPredictions = AvoTest[features]
XforPredictions.head()

,Date,TotalVolume,DataBatch,type,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region
id,,,,,,,,,,,,,
0,736176,28969.34,21,1,80.77,27361.91,0.00,1526.66,1526.66,0.00,0.00,1,36
1,736561,322962.89,19,0,5632.85,242365.82,2441.56,72522.66,22942.48,46083.51,3496.67,0,22
2,736351,155334.45,49,1,3657.79,74068.65,0.00,77608.01,27302.29,50305.72,0.00,1,43
3,735875,10231.74,12,1,341.89,8519.00,0.00,1370.85,303.33,1067.52,0.00,1,51
4,735889,10652.02,10,1,6905.95,39.52,0.00,3706.55,3706.55,0.00,0.00,1,32


In [16]:
# Make Predictions!
yPredictions = model.predict(XforPredictions)
print("Predictions:", yPredictions)

Predictions: [1.94058 1.53467 1.27191 ... 1.49494 1.75636 1.57191]


In [17]:
# Save to CSV File!
output = pd.DataFrame({'id': XforPredictions.index, 'AveragePrice': yPredictions})
output['AveragePrice'] = [f"{x:.2f}" for x in output["AveragePrice"]]
output.to_csv('avocado-submission.csv', index=False)
print(output)

        id AveragePrice
0        0         1.94
1        1         1.53
2        2         1.27
3        3         1.56
4        4         1.44
...    ...          ...
5470  5470         1.74
5471  5471         1.39
5472  5472         1.49
5473  5473         1.76
5474  5474         1.57

[5475 rows x 2 columns]


# Upload for grading

In [18]:
# Download CTFSG Grader Libraries
import urllib.request, os
urllib.request.urlretrieve('https://raw.githubusercontent.com/alttablabs/ctfsg-utils/master/pyctfsglib.py', './pyctfsglib.py')
print('Downloaded pyctfsglib.py:', 'pyctfsglib.py' in os.listdir())

Downloaded pyctfsglib.py: True


In [19]:
# Connect to graders
import pyctfsglib as ctfsg
import random

USER_TOKEN = "bsmZpjHmOHygqXWFAPNdoXgJKUQbrxBsYDVOddoNCNTsjlZRJMhmSEiVrOHqAauL" # You need to fill this up
GRADER_URL = random.choice([
"http://challenges.csdc20t.ctf.sg:30001/", "http://challenges.csdc20t.ctf.sg:30002/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm AVOCADO_PRICE GRADER_5021d39ae191


In [20]:
grader.submitFile('avocado-submission.csv')

ProofOfWork Challenge =>  ('CTFSGRB6cee287bbb295a96cd027dc4f1df1e10', 22)
ProofOfWork Answer Found! =>  2346607


'{"challenge":{"name":"Avocado Prices"},"id":"ck8q2xpfla2gi0726b5madqb9","status":"PARTIALLY_CORRECT","multiplier":0.8941,"submittedBy":{"username":"nyjc-3"},"createdAt":"2020-04-07T15:49:53Z"}'

In [22]:
0.8941*500

447.05